In [1]:
import os
import sys
import csv
from enum import Enum

import numpy as np
import pandas as pd

# For Feature Extraction
import tsfresh as tsf
sys.path.append('../src')
import anoog
import time

In [2]:
df = anoog.io.load_data(data_path='../data/2021-11-09', 
                        persons=['skadkade', 'vkorzev'], 
                        extraction=anoog.io.extraction_mode.NONE,
                        show_extraction=False,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False) 
df=df.drop(columns = ["y"])
df

,Time,Audio,Voltage,Current,ID
0,2021-11-09 11:51:30.000,-1.484375,20.408691,-0.719727,0.0
1,2021-11-09 11:51:30.010,3.084775,20.402274,-0.807125,0.0
2,2021-11-09 11:51:30.020,-1.039690,20.401973,-0.796764,0.0
3,2021-11-09 11:51:30.030,-1.501513,20.401470,-0.800560,0.0
4,2021-11-09 11:51:30.040,3.164062,20.402350,-0.803417,0.0
...,...,...,...,...,...
24137,2021-11-09 12:24:46.560,-0.077687,19.553144,-0.838514,24.0
24138,2021-11-09 12:24:46.570,0.538470,19.554451,-0.827846,24.0
24139,2021-11-09 12:24:46.580,-0.509043,19.554166,-0.829795,24.0
24140,2021-11-09 12:24:46.590,0.256057,19.555244,-0.832509,24.0


In [3]:
#Konvertierung von timestamp zu String
time = []
for i in df["Time"]:
    time.append(str(i))

In [4]:
zeit = []
for i in time:
    zeit.append(i[11:])

stunden = []
minuten = []
sekunden = []
for i in zeit:
    stunden.append(i[:2])
    minuten.append(i[3:5])
    sekunden.append(i[6:])

#Konvertierung zu Sekunden
sec = []

for st in stunden:
    sec.append(int(st)*3600)
z = 0
for mi in minuten:
    sec[z]=sec[z]+(int(mi)*60)
    z +=1
z = 0
for se in sekunden:
    sec[z]=sec[z]+float(se)
    z +=1
df["Time"] = sec

In [5]:
current = []
z=1
num = 0
current_mean = 0
for i in df["ID"].unique():
    for j,value in enumerate(df["Current"]):
        if df["ID"][j] == i:
            current_mean += value
            num+=1
    current += [current_mean/num]
    current_mean = 0
    num = 0
    
len(current)

50

In [6]:
zeit = []
z=1
num = 0
Zeit = []
for i in df["ID"].unique():
    for j,value in enumerate(df["Time"]):
        if df["ID"][j] == i:
            Zeit += [value]
            num+=1
    zeit += [Zeit[-1]-Zeit[0]]
    Zeit = []
    num = 0

len(zeit)

50

In [7]:
voltage = []
z=1
num = 0
voltage_mean = 0
for i in df["ID"].unique():
    for j,value in enumerate(df["Voltage"]):
        if df["ID"][j] == i:
            voltage_mean += value
            num+=1
    voltage += [voltage_mean/num]
    voltage_mean = 0
    num = 0
    
len(voltage)

50

In [8]:
audio = []
z=1
num = 0
audio_mean = 0
for i in df["ID"].unique():
    for j,value in enumerate(df["Audio"]):
        if df["ID"][j] == i:
            audio_mean += value
            num+=1
    audio += [audio_mean/num]
    audio_mean = 0
    num = 0

len(audio)

50

In [9]:
df

,Time,Audio,Voltage,Current,ID
0,42690.00,-1.484375,20.408691,-0.719727,0.0
1,42690.01,3.084775,20.402274,-0.807125,0.0
2,42690.02,-1.039690,20.401973,-0.796764,0.0
3,42690.03,-1.501513,20.401470,-0.800560,0.0
4,42690.04,3.164062,20.402350,-0.803417,0.0
...,...,...,...,...,...
24137,44686.56,-0.077687,19.553144,-0.838514,24.0
24138,44686.57,0.538470,19.554451,-0.827846,24.0
24139,44686.58,-0.509043,19.554166,-0.829795,24.0
24140,44686.59,0.256057,19.555244,-0.832509,24.0


In [10]:
a = np.arange(25)
b = np.arange(25)
ID=np.concatenate((a, b))
d = {"Time" : zeit,
         "Audio" : audio,
         "Voltage" : voltage,
         "Current" : current,
         "ID" : ID}
df_new = pd.DataFrame(data=d)

In [11]:
df_new

,Time,Audio,Voltage,Current,ID
0,9.46,-0.001055,19.798838,2.515298,0
1,10.99,0.437867,19.742964,2.916861,1
2,4.21,0.002595,19.670406,3.256249,2
3,3.27,0.001846,19.555644,3.927147,3
4,7.24,0.013270,19.666646,2.803406,4
5,3.06,-0.007539,19.400382,4.524385,5
6,3.02,-0.030464,19.465499,4.007386,6
7,3.25,0.002827,19.426912,4.307221,7
8,3.47,-0.003164,19.453496,3.951887,8
9,6.95,0.001098,19.671461,2.386083,9


In [12]:
# Berechnung vom Widerstand Ohm
df_new["Resistance"] = df_new["Current"] / df_new["Voltage"]

# Berechnung der Elektrischen Leistung P in Joule
df_new["Power"] = df_new["Current"] * df_new["Voltage"]

# Berechnung der Arbeit W
df_new["Work"] = df_new["Current"] * df_new["Voltage"] * df_new["Time"]

# Berechnung von mAh(milli Ampere Stunden)
df_new["mAh"] = (df_new["Current"] / 60) * 1000

# Berechnung von Wh(Watt Stunden)
df_new["Wh"] = df_new["mAh"] * df_new["Voltage"]

df_new

,Time,Audio,Voltage,Current,ID,Resistance,Power,Work,mAh,Wh
0,9.46,-0.001055,19.798838,2.515298,0,0.127043,49.799973,471.107740,41.921629,829.999542
1,10.99,0.437867,19.742964,2.916861,1,0.147742,57.587490,632.886513,48.614356,959.791496
2,4.21,0.002595,19.670406,3.256249,2,0.165541,64.051746,269.657849,54.270821,1067.529093
3,3.27,0.001846,19.555644,3.927147,3,0.200819,76.797891,251.129103,65.452452,1279.964849
4,7.24,0.013270,19.666646,2.803406,4,0.142546,55.133593,399.167214,46.723432,918.893217
5,3.06,-0.007539,19.400382,4.524385,5,0.233211,87.774803,268.590897,75.406423,1462.913383
6,3.02,-0.030464,19.465499,4.007386,6,0.205871,78.005762,235.577402,66.789762,1300.096036
7,3.25,0.002827,19.426912,4.307221,7,0.221714,83.676001,271.947004,71.787016,1394.600020
8,3.47,-0.003164,19.453496,3.951887,8,0.203145,76.878010,266.766696,65.864778,1281.300173
9,6.95,0.001098,19.671461,2.386083,9,0.121297,46.937741,326.217300,39.768052,782.295684
